In [18]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


## Importing Dataset

In [2]:
# reading data train

data_train = pd.read_csv("train.csv")

In [3]:
data_train.head()

,age,education,default,balance,housing,loan,day,month,duration,campaign,...,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,0.695517,-0.232452,0,-0.713083,1,0,0.695517,-0.401396,-0.260866,-0.900753,...,0,0,0,0,0,0,0,0,1,0
1,-0.636765,-0.232452,0,-0.761484,1,1,-0.636765,-0.401396,-1.015248,-0.900753,...,0,1,0,0,0,0,0,0,0,0
2,1.058867,-0.232452,0,1.934648,1,0,1.058867,-0.401396,-0.854313,-0.900753,...,1,0,0,0,0,0,0,0,0,0
3,-0.636765,-0.232452,0,-0.763277,0,0,-0.636765,-0.401396,0.211880,-0.900753,...,1,0,0,0,0,0,0,0,0,0
4,-0.394532,1.353897,0,-0.350969,1,0,-0.394532,-0.401396,-0.381567,-0.900753,...,0,0,0,1,0,0,0,0,0,0


In [4]:
# reading data test

data_test = pd.read_csv("test1.csv")

In [5]:
data_test.head()

,age,education,default,balance,housing,loan,day,month,duration,campaign,...,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,-1.056270,-1.758874,0,0.121072,0,0,-1.056270,1.611873,-0.711861,-0.576829,...,0,0,0,0,0,0,0,0,0,1
1,-0.772583,-0.227607,0,1.118644,1,1,-0.772583,-0.490631,-0.169194,-0.576829,...,0,0,0,0,0,0,1,0,0,0
2,-0.583458,1.303660,0,-0.024144,1,0,-0.583458,-0.911131,-0.303898,-0.576829,...,0,0,0,1,0,0,0,0,0,0
3,-1.056270,1.303660,0,0.017726,1,1,-1.056270,-0.070130,-0.250017,0.387967,...,0,0,0,1,0,0,0,0,0,0
4,1.686036,-0.227607,0,-0.472753,1,0,1.686036,-0.490631,-0.146102,-0.576829,...,1,0,0,0,0,0,0,0,0,0


## Modelling

### Split Data Train & Test

#### Kaggle Default

In [6]:
# splitting by kaggle task default

jumlah_train = data_train.shape[0]
jumlah_test  = data_test.shape[0]

total      = jumlah_train + jumlah_test
perc_train = round(jumlah_train/total*100,2)
perc_test  = round(jumlah_test/total*100,2)

print("Jumlah Train Dataset adalah sejumlah : " +str(jumlah_train)+" ("+str(perc_train)+"%)")
print("Jumlah Test Dataset adalah sejumlah  : " +str(jumlah_test)+" ("+str(perc_test)+"%)")

Jumlah Train Dataset adalah sejumlah : 24097 (84.2%)
Jumlah Test Dataset adalah sejumlah  : 4521 (15.8%)


In [9]:
# categorising feature & label
feature = ['age','education','default','balance','housing','loan','day','month','duration','campaign','previous',
          'contact_cellular','contact_telephone','marital_divorced','marital_married','marital_single',
          'job_admin.','job_blue-collar','job_entrepreneur','job_housemaid','job_management','job_retired','job_self-employed',
          'job_services','job_student','job_technician','job_unemployed']
label = ['y']

In [11]:
# train variable

X_train = data_train[feature]
y_train = data_train['y']

In [12]:
# test variabel

X_test = data_test[feature]
y_test = data_test['y']

### Modeling Method

#### Logistic Regresion

In [26]:
logreg = LogisticRegression(solver='lbfgs', max_iter=3000)

In [27]:
logreg.fit(X_train, y_train)

LogisticRegression(max_iter=3000)

In [28]:
y_predicted = logreg.predict(X_test) # prediksi data test

In [29]:
print(round(accuracy_score(y_test, y_predicted),2))# contoh evaluasi dengan akurasi

0.89


#### Decision Tree

### Model Evaluation

### Precision

### Hyperparameter Tuning

## Feature Importance

### Most Important Feature

### Business Insight